In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pylab import *
from PIL import Image
import os
import cv2
from keras import preprocessing
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from skimage import exposure
import imageio

In [2]:
trainingDataHEM = 'C:/Users/v4run/OneDrive/Desktop/Datasets/C-NMC_Leukemia/training_data/fold_1/cropped_hem'
trainingDataALL = 'C:/Users/v4run/OneDrive/Desktop/Datasets/C-NMC_Leukemia/training_data/fold_1/cropped_all'
trainingDataAugHEM = 'C:/Users/v4run/OneDrive/Desktop/Datasets/C-NMC_Leukemia/training_data/fold_1/aug_hem'

validationDataHEM = 'C:/Users/v4run/OneDrive/Desktop/Datasets/C-NMC_Leukemia/training_data/fold_0/hem'
validationDataALL = 'C:/Users/v4run/OneDrive/Desktop/Datasets/C-NMC_Leukemia/training_data/fold_0/all'

In [3]:
def get_imlist(path):
    return [os.path.join(path,f) for f in os.listdir(path) if f.endswith('.bmp')]

In [4]:
def crop_img(images, display=False):
    mask = images == 0

    coords = np.array(np.nonzero(~mask))
    top_left = np.min(coords, axis=1)
    bottom_right = np.max(coords, axis=1)

    cropped_image = images[top_left[0]:bottom_right[0],top_left[1]:bottom_right[1]]
    return cropped_image

In [5]:
def dataImport(path,pathType,dataType):
    imageList = []
    for dtype in dataType:
        if dtype == 'Validation':
            for i,j in zip(pathType,path):
                if i not in ['ALL','HEM']:
                    raise ValueError("Entry must be either 'ALL' or 'HEM'")
                else:
                    if i == 'HEM':
                        loadPath = get_imlist(j) 
                        for paths in loadPath:
                            imageList.append({'Images':Image.open(paths),'Labels':0,'CroppedImages':None})
                    else:
                        loadPath = get_imlist(j) 
                        for paths in loadPath:
                            imageList.append({'Images':Image.open(paths),'Labels':1,'CroppedImages':None})
            imageData = pd.DataFrame(imageList)
            imageData['CroppedImages'] = imageData['Images'].apply(lambda images: crop_img(np.array(images)))
        else:
            for i,j in zip(pathType,path):
                if i not in ['ALL','HEM']:
                    raise ValueError("Entry must be either 'ALL' or 'HEM'")
                else:
                    if i == 'HEM':
                        loadPath = get_imlist(j) 
                        for paths in loadPath:
                            imageList.append({'Images':Image.open(paths),'Labels':0})
                    else:
                        loadPath = get_imlist(j) 
                        for paths in loadPath:
                            imageList.append({'Images':Image.open(paths),'Labels':1})

            imageData = pd.DataFrame(imageList)
            print('Imdone...')
    return imageData

In [6]:
def load_images_labels_from_df(path, pathType, dataType, img_size):

    df = dataImport(path,pathType,dataType)
    images = []
    labels = []
    for dType in dataType:
        if dType == 'Validation':
            for _, row in df.iterrows():
                label = row['Labels']
                img_data = row['CroppedImages']
                img_resized = cv2.resize(img_data, (img_size, img_size), interpolation=cv2.INTER_LANCZOS4) #resize images (128*128, 256*256 etc)
                img_array = exposure.equalize_adapthist(img_resized,clip_limit=0.15,nbins=256)  # Normalize pixel values
                img_array = (img_array*255).astype(np.uint8)
                images.append(img_array)
                labels.append(label)
            
        else:
            print('Imhere...')
            for _, row in df.iterrows():
                label = row['Labels']
                img_data = np.array(row['Images'])
                images.append(img_data)
                labels.append(label)

    images = np.array(images)
    labels = np.array(labels)
    
    return images, labels, len(images)

In [7]:
# Training Dataset
imageTrainData, labelTrainData, totalTrainData = load_images_labels_from_df([trainingDataALL,trainingDataHEM,trainingDataAugHEM],
                                                                  ['ALL','HEM','HEM'],['Train','Train','Train'],256)

Imdone...


In [7]:
imageValidData, labelValidData, totalValidData = load_images_labels_from_df([validationDataALL,validationDataHEM],
                                                                  ['ALL','HEM'],['Validation','Validation'],256)